<a href="https://colab.research.google.com/github/ykato27/Metric-Learning/blob/main/PyTroch_Metric_Learning_DistributedTripletMarginLossMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytorch-metric-learning
!pip install faiss-cpu # we're using cpu for this example

     |████████████████████████████████| 105 kB 4.2 MB/s 
     |████████████████████████████████| 8.4 MB 4.3 MB/s 


In [2]:
######################################################################################################################
### This script is modified from the guide on pytorch distributed training https://github.com/seba-1511/dist_tuto.pth/
### https://pytorch.org/tutorials/intermediate/dist_tuto.html
######################################################################################################################
import os
import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from math import ceil
from random import Random
from torch.multiprocessing import Process
from torchvision import datasets, transforms

from pytorch_metric_learning.utils import distributed as pml_dist
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning import losses, miners, testers
import numpy as np
import logging
logging.getLogger().setLevel(logging.INFO)


class Partition(object):
    """ Dataset-like object, but only access a subset of it. """

    def __init__(self, data, index):
        self.data = data
        self.index = index

    def __len__(self):
        return len(self.index)

    def __getitem__(self, index):
        data_idx = self.index[index]
        return self.data[data_idx]


class DataPartitioner(object):
    """ Partitions a dataset into different chuncks. """

    def __init__(self, data, sizes=[0.7, 0.2, 0.1], seed=1234):
        self.data = data
        self.partitions = []
        rng = Random()
        rng.seed(seed)
        data_len = len(data)
        indexes = [x for x in range(0, data_len)]
        rng.shuffle(indexes)

        for frac in sizes:
            part_len = int(frac * data_len)
            self.partitions.append(indexes[0:part_len])
            indexes = indexes[part_len:]

    def use(self, partition):
        return Partition(self.data, self.partitions[partition])


class Net(nn.Module):
    """ Network architecture. """

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        return self.fc1(x)

def get_MNIST(train):
    return datasets.MNIST(
            './data',
            train=train,
            download=True,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307, ), (0.3081, ))
            ]))
    

def partition_dataset(dataset):
    """ Partitioning MNIST """
    size = dist.get_world_size()
    bsz = 512 // size
    partition_sizes = [1.0 / size for _ in range(size)]
    partition = DataPartitioner(dataset, partition_sizes)
    partition = partition.use(dist.get_rank())
    train_set = torch.utils.data.DataLoader(
        partition, batch_size=bsz, shuffle=True)
    return train_set, bsz


### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model, data_device):
    # dataloader_num_workers has to be 0 to avoid pid error
    # This only happens when within multiprocessing
    tester = testers.BaseTester(dataloader_num_workers=0, data_device=data_device)
    return tester.get_all_embeddings(dataset, model)


### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator, data_device):
    train_embeddings, train_labels = get_all_embeddings(train_set, model, data_device)
    test_embeddings, test_labels = get_all_embeddings(test_set, model, data_device)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(test_embeddings, 
                                                train_embeddings,
                                                test_labels,
                                                train_labels,
                                                False)
    print("Validation set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))

def test_model(rank, train_set, test_set, model, epoch, data_device):
    if rank == 0:
        print("Computing validation set accuracy for epoch {}".format(epoch))
        accuracy_calculator = AccuracyCalculator(include = ("precision_at_1",), k = 1)
        test(train_set, test_set, model, accuracy_calculator, data_device)
    dist.barrier()


def run(rank, size, train_dataset, val_dataset):
    """ Distributed Synchronous SGD Example """
    print("Rank {} entering the 'run' function".format(rank))
    torch.manual_seed(1234)
    train_set, bsz = partition_dataset(train_dataset)
    dist.barrier()
    ### use this if you have multiple GPUs ###
    # device = torch.device("cuda:{}".format(rank))
    device = torch.device("cpu")
    model = Net()
    ### if you have multiple GPUs, set this to DDP(model.to(device), device_ids=[rank])
    model = DDP(model.to(device))
    test_model(rank, train_dataset, val_dataset, model, "untrained", device)

    optimizer = optim.Adam(model.parameters(), lr=0.01)

    #####################################
    ### pytorch-metric-learning stuff ###
    loss_fn = losses.TripletMarginLoss()
    ### if you have multiple GPUs, set this to pml_dist.DistributedLossWrapper(loss=loss_fn, device_ids=[rank])
    loss_fn = pml_dist.DistributedLossWrapper(loss=loss_fn)
    miner = miners.MultiSimilarityMiner()
    miner = pml_dist.DistributedMinerWrapper(miner=miner)
    ### pytorch-metric-learning stuff ###
    #####################################

    num_batches = ceil(len(train_set.dataset) / float(bsz))
    for epoch in range(1):
        epoch_loss = 0.0
        print("Rank {} starting epoch {}".format(rank, epoch))
        for i, (data, target) in enumerate(train_set):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            hard_pairs = miner(output, target)
            loss = loss_fn(output, target, hard_pairs)
            epoch_loss += loss.item()
            loss.backward()
            optimizer.step()
            if i % 10 == 0:
                print('Rank {}, iteration {}, loss {}, num pos pairs {}, num neg pairs {}'.\
                      format(rank, i, loss.item(), miner.miner.num_pos_pairs, miner.miner.num_neg_pairs))
            dist.barrier()

        print('Rank {}, epoch {}, average loss {}'.format(rank, epoch, epoch_loss/num_batches))
        test_model(rank, train_dataset, val_dataset, model, epoch, device)



#######################################
### Set backend='nccl' if using GPU ###
#######################################
def init_processes(rank, size, fn, train_dataset, val_dataset, backend='gloo'):
    """ Initialize the distributed environment. """
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '29500'
    dist.init_process_group(backend, rank=rank, world_size=size)
    fn(rank, size, train_dataset, val_dataset)


if __name__ == "__main__":
    train_dataset = get_MNIST(True)
    val_dataset = get_MNIST(False)

    size = 4
    processes = []
    for rank in range(size):
        p = Process(target=init_processes, args=(rank, size, run, train_dataset, val_dataset))
        p.start()
        processes.append(p)

    for p in processes:
        p.join()

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Rank 0 entering the 'run' function
Rank 2 entering the 'run' function
Rank 1 entering the 'run' function
Rank 3 entering the 'run' function
Computing validation set accuracy for epoch untrained


  0%|          | 0/1875 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 313/313 [00:03<00:00, 96.41it/s]


Computing accuracy
Validation set accuracy (Precision@1) = 0.9228
Rank 3 starting epoch 0
Rank 0 starting epoch 0
Rank 1 starting epoch 0
Rank 2 starting epoch 0


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use